In [1]:
from transformers import pipeline
from transformers import AutoProcessor, SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio
import torch
from IPython.display import Image
from nltk.translate.bleu_score import sentence_bleu
import openai
import requests 
import os
import evaluate
import warnings

In [2]:
url = 'https://i.imgur.com/1z7t7vR.jpg' 
url2 = 'https://i.imgur.com/pyCnc.jpg' 
url3 = 'http://i.imgur.com/FIf0r.jpg' 

In [3]:
vit_gpt2 = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/wjkoh005/.local/lib/python3.8/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [4]:
blip_base = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")

In [5]:
blip_2 = pipeline("image-to-text", model="Salesforce/blip2-opt-2.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#git_large = pipeline("image-to-text", model="microsoft/git-large-coco")

In [7]:
# Coco Dataset
URL1 = 'http://farm6.staticflickr.com/5295/5533938581_f87f4c59e2_z.jpg'
URL2 = 'http://farm9.staticflickr.com/8035/8072572904_ee0028185d_z.jpg'
URL3 = 'http://farm7.staticflickr.com/6150/5931568092_987f4608b0_z.jpg'

In [8]:
Image(url=URL1 , width=400, height=300)

In [9]:
Image(url=URL2 , width=400, height=300)

In [10]:
Image(url=URL3 , width=400, height=300)

In [11]:
vit_gpt2(URL1, max_new_tokens=100)[0]['generated_text']

'a refrigerator freezer sitting in a kitchen next to a counter '

In [12]:
vit_gpt2_output = vit_gpt2(URL1, max_new_tokens=100)[0]['generated_text']

In [13]:
vit_gpt2(URL2, max_new_tokens=100)[0]['generated_text']

'a bench on a mountain with a view of the mountains '

In [14]:
vit_gpt2(URL3, max_new_tokens=100)[0]['generated_text']

'a large jetliner flying through a cloudy sky '

In [15]:
blip_base(URL1, max_new_tokens=100)[0]['generated_text']

'a white refrigerator sitting in a kitchen next to a sink'

In [16]:
blip_base_output = blip_base(URL1, max_new_tokens=100)[0]['generated_text']

In [17]:
blip_base(URL2, max_new_tokens=100)[0]['generated_text']

'a bench on a rocky mountain with a view of a mountain'

In [18]:
blip_base(URL3, max_new_tokens=100)[0]['generated_text']

'a plane flying in the sky'

In [19]:
blip_2(URL1, max_new_tokens=100)[0]['generated_text']

'a refrigerator sitting in a kitchen with cabinets\n'

In [20]:
blip_2_output = blip_2(URL1, max_new_tokens=100)[0]['generated_text']

In [21]:
blip_2(URL2, max_new_tokens=100)[0]['generated_text']

'a bench sitting on a hill\n'

In [22]:
blip_2(URL3, max_new_tokens=100)[0]['generated_text']

'a large jetliner flying through the air\n'

In [23]:
# Calculating BLEU Score using the frige example
reference = [
             ['this', 'is', 'the', 'back', 'of', 'a', 'freezer', 'in', 'the', 'middle', 'of', 'a', 'kitchen', 'floor'],
             ['the', 'appliance', 'in', 'the', 'kitchen', 'may', 'be', 'broken', '.'],
             ['the', 'back', 'of', 'a', 'refrigerator', 'in', 'a', 'kitchen'],
             ['a', 'refrigerator', 'sitting', 'in', 'the', 'middle', 'of', 'a', 'kitchen', 'floor', '.'],
             ['the', 'back', 'of', 'a', 'refrigerator', 'standing', 'in', 'the', 'middle', 'of', 'a', 'kitchen', '.'],
            ]

In [24]:
vit_candidate = ['a', 'refrigerator', 'freezer', 'sitting', 'in', 'a', 'kitchen', 'next', 'to', 'a', 'counter ']

In [25]:
vit_score = sentence_bleu(reference, vit_candidate)
print(vit_score)

5.008605395783359e-78


/home/wjkoh005/.local/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [26]:
print('Cumulative 1-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, vit_candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.636364
Cumulative 2-gram: 0.504525
Cumulative 3-gram: 0.308321
Cumulative 4-gram: 0.000000


In [27]:
blip_base_candidate = ['a', 'white', 'refrigerator', 'sitting', 'in', 'a', 'kitchen', 'next', 'to', 'a', 'sink']

In [28]:
blip_base_score = sentence_bleu(reference, blip_base_candidate)
print(blip_base_score)

5.731095145962094e-78


In [29]:
print('Cumulative 1-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, blip_base_candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.545455
Cumulative 2-gram: 0.467099
Cumulative 3-gram: 0.368341
Cumulative 4-gram: 0.000000


In [30]:
blip_2_candidate = ['a', 'refrigerator', 'sitting', 'in', 'a', 'kitchen', 'with', 'cabinets\n']

In [31]:
blip_2_score = sentence_bleu(reference, blip_2_candidate)
print(blip_2_score)

0.4810977290978808


In [32]:
print('Cumulative 1-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, blip_2_candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.750000
Cumulative 2-gram: 0.731925
Cumulative 3-gram: 0.647453
Cumulative 4-gram: 0.481098


In [33]:
openai.api_key = "" # Input your personal openai key within the "" 

In [34]:
conversation_history = []

In [35]:
def chat_with_gpt(message):
    conversation_history.append("User: " + message)
    
    response = openai.Completion.create(
        engine="text-davinci-003",  # Choose the appropriate engine
        prompt="\n".join(conversation_history),
        temperature=0.7,
        max_tokens=2049
    )
    
    reply = response.choices[0].text.strip()
    
    prefixes_to_remove = ["Answer:", "AI:"]
    for prefix in prefixes_to_remove:
        if reply.startswith(prefix):
            reply = reply[len(prefix):].strip()
    
    conversation_history.append("AI: " + reply)
    
    return reply

In [36]:
vit_gpt2_output

'a refrigerator freezer sitting in a kitchen next to a counter '

In [37]:
blip_base_output

'a white refrigerator sitting in a kitchen next to a sink'

In [38]:
user_message1 = "Combine both text:" + vit_gpt2_output + "and " + blip_base_output
experiment1 = chat_with_gpt(user_message1)

In [39]:
print(experiment1)

A white refrigerator freezer is sitting in a kitchen next to a counter and a sink.


In [40]:
user_message2 = "Combine and refine the text:" + vit_gpt2_output + "and " + blip_base_output
experiment2 = chat_with_gpt(user_message2)

In [41]:
print(experiment2)

A white refrigerator freezer is situated in a kitchen next to both a counter and a sink.


In [42]:
user_message3 = "Combine both:" + vit_gpt2_output + "and " + blip_base_output + " Please refine the text."
experiment3 = chat_with_gpt(user_message3)

In [43]:
print(experiment3)

A white refrigerator freezer is positioned in a kitchen near a counter and sink.


In [45]:
user_message4 = "You are an AI assistant that combines information. The following are the outputs:" + vit_gpt2_output + "and "+ blip_base_output + " Please refine the text."
experiment4 = chat_with_gpt(user_message4)

In [46]:
print(experiment4)

A white refrigerator freezer is situated in a kitchen close to both a counter and a sink.


In [47]:
user_message5 = "Act as a human, provide output that a human would say to a visually impaired from the combination of both text:" + vit_gpt2_output + "and " + blip_base_output
experiment5 = chat_with_gpt(user_message5)

In [48]:
print(experiment5)

There is a white refrigerator freezer in the kitchen next to both a counter and a sink.


In [49]:
user_message6 = "Act as a human, be as elaborative as possible, combined and refined the text output that a human would say to a visually impaired:" + vit_gpt2_output + "and " + blip_base_output
experiment6 = chat_with_gpt(user_message6)

In [50]:
print(experiment6)

A white refrigerator freezer is placed in the kitchen near a counter and a sink, providing a convenient storage solution for food and other items.


In [51]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [53]:
inputs = processor(text=experiment6, return_tensors="pt")

In [54]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

In [55]:
speaker_embeddings.shape

torch.Size([1, 512])

In [56]:
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [57]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [58]:
speech.shape

torch.Size([157696])

In [59]:
Audio(speech, rate=16000)